# Tic Tac Toe
Alpha-Beta Search

In [7]:
import copy

class Game():
    player_max = 'Max'
    player_min = 'Min'
    player_none = 'None'

    def __init__(self, num_rows=3, num_columns=3, consec_threshold=3):
        self.num_rows = num_rows
        self.num_columns = num_columns
        self.consec_threshold = consec_threshold  # threshold for consecutive

        self.initial_state = [[self.player_none] * num_columns for i in range(num_rows)]

    def __repr__(self):
        output = ''
        for row in self.initial_state:
            for element in row:
                output += element + ' '
        
        return output

    def _count(self, state, player):
        # count cells of player
        return sum([sum([element == player for element in row]) for row in state])

    def to_move(self, state):
        cell_count = {}
        cell_count[self.player_max] = self._count(state, self.player_max)
        cell_count[self.player_min] = self._count(state, self.player_min)
        cell_count[self.player_none] = self._count(state, self.player_none)

        # determine which player makes the first move
        first_move_player = self.player_max
        second_move_player = self.player_min
        if cell_count[first_move_player] <= cell_count[second_move_player]:
            player_to_move = first_move_player
        else:
            player_to_move = second_move_player

        return player_to_move
    
    def is_terminal(self, state):
        return self.utility(state, self.player_max) is not None
    
    def actions(self, state):
        actions = []
        for row in range(self.num_rows):
            for column in range(self.num_columns):
                if state[row][column] == self.player_none:
                    action = (row, column)
                    actions.append(action)
        
        return actions
    
    def result(self, state, action):
        row = action[0]
        column = action[1]
        player = self.to_move(state)

        state = copy.deepcopy(state)
        state[row][column] = player
        return state
    
    def _get_down_diag_starts(self):
        start_coords = []
        for row_index in range(self.num_rows - self.consec_threshold, -1, -1):
            start_coord = (row_index, 0)
            start_coords.append(start_coord)
        
        for column_index in range(1, self.num_columns - self.consec_threshold + 1):
            start_coord = (0, column_index)
            start_coords.append(start_coord)
        
        return start_coords

    def _get_winner(self, cells):
        last_cell = None
        counter = 0
        for player in cells:
            if player != last_cell:
                last_cell = player
                counter = 1
            else:
                counter += 1

            if counter >= self.consec_threshold:
                return player
        
        return None

    def utility(self, state, player):
        """Get the utility of the current state. This function is rather overengineered.
        The consecutive threshold is variable. For example a consecutive threshold of 3
        in a 5x6 grid works correctly, even if multiple winners exist in the current
        game state. :')"""
        winner = []

        # calculate winners in the rows
        for row in state:
            winner.append(self._get_winner(row))
            
        # calculate winners in the columns
        for column_index in range(self.num_columns):
            column = [state[row_index][column_index] for row_index in range(self.num_rows)]
            winner.append(self._get_winner(column))

        # calculate winners in the downward diagonals
        down_diag_starts = self._get_down_diag_starts()
        for down_diag_start in down_diag_starts:
            down_diag = []
            row, column = down_diag_start
            while row < self.num_rows and column < self.num_columns:
                down_diag.append(state[row][column])
                row += 1
                column += 1
            winner.append(self._get_winner(down_diag))

        # flip column to get upward diagonal starts
        up_diag_starts = [(self.num_rows - r_c[0] - 1, r_c[1])
                          for r_c in down_diag_starts]
        # calculate winners in the upward diagonals
        for up_diag_start in up_diag_starts:
            up_diag = []
            row, column = up_diag_start
            while row < self.num_rows and column < self.num_columns:
                up_diag.append(state[row][column])
                row -= 1
                column += 1
            winner.append(self._get_winner(up_diag))
        
        if self._count(state, self.player_none) != 0 and (winner.count(self.player_max) == 0 and winner.count(self.player_min) == 0):
            return None
        
        player_max_utility = winner.count(self.player_max) * 1
        player_min_utility = winner.count(self.player_min) * (-1)
        #player_none_utility = winner.count(self.player_none) * 0

        utility_sum = player_max_utility + player_min_utility# + player_none_utility
        if player == self.player_min:
            utility_sum *= -1
        
        return utility_sum

In [8]:
from search import alpha_beta_search

def draw(state):
    print(''.join([''.join([{'Max': 'x ','Min': 'o ', 'None': '- '}[element] for element in row]) + '\n' for row in state]))

game = Game()
state = game.initial_state
draw(state)
while not game.is_terminal(state):
    state = game.result(state, alpha_beta_search(game, state))
    draw(state)

- - - 
- - - 
- - - 

x - - 
- - - 
- - - 

x - - 
- o - 
- - - 

x x - 
- o - 
- - - 

x x o 
- o - 
- - - 

x x o 
- o - 
x - - 

x x o 
o o - 
x - - 

x x o 
o o x 
x - - 

x x o 
o o x 
x o - 

x x o 
o o x 
x o x 

